In [1]:
# Keeping this class here for now (easy to test). Will move this to a separate .py file when we do code-cleanup/ submission

import psycopg2
!{psycopg2.executable} -m pip install psycopg2
import psycopg2.extras
import re
import pandas


class FoodViolationData:

    def __init__(self, connection_string):
        self.conn = psycopg2.connect(connection_string)

    class FoodViolationQuery:

        def __init__(self, product_data):
            self.joins = []
            self.conditions = []
            self.parameters = {}
            self.product_data = product_data

            self.named_params = {}

        def __param_index(self, param_name):

            if param_name not in self.named_params:
                self.named_params.update({param_name: 0})
                return 0
            else:
                i = self.named_params[param_name] + 1
                self.named_params.update({param_name: i})
                return i
        
        def searchByCounty(self, name):
            i = str(self.__param_index("county"))
            self.conditions.append(""" "COUNTY" like %(county""" + i + ")s")
            #print(i)
            #print(name)
            #self.conditions.append(""" "COUNTY" like '%%""" + name + """%%' """)
            #print(self.conditions)
            self.parameters.update({"county" + i: "%" + name + "%"})
            #print(self.parameters)
            return self

        def searchByOperatorID(self, health_opID):
            i = str(self.__param_index("health_operation_id"))
            self.conditions.append(""" "NYS HEALTH OPERATION ID" = %(health_operation_id""" + i + ")s")
            #print(i)
            #print(name)
            #self.conditions.append(""" "COUNTY" like '%%""" + name + """%%' """)
            #print(self.conditions)
            self.parameters.update({"health_operation_id" + i: health_opID})
            #print(self.parameters)
            return self

        def __compile(self):
            query = "SELECT * from food_service_inspections"
            query = query + " " + " ".join(self.joins) if self.joins else query
            query = query + " WHERE " + " AND ".join(self.conditions) if self.conditions else query
            #query += " ORDER BY food_service_inspections.code"
            return query

        def run(self):
            query = self.__compile()
            #query = """select * from food_service_inspections where "COUNTY" like 'ALB%%' limit 3"""
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                print(query)
                cursor.execute(query, self.parameters)
                return pandas.DataFrame(cursor.fetchall())

        def show(self):
            query = self.__compile()
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                return cursor.mogrify(query, self.parameters)

    def query(self):
        return self.FoodViolationQuery(self)

ModuleNotFoundError: No module named 'psycopg2'

In [9]:
import re
import pandas
import matplotlib

ModuleNotFoundError: No module named 'pandas'

In [5]:
fd_vio = FoodViolationData("host='localhost' dbname='test5' user='test5' password='test5'")

NameError: name 'FoodViolationData' is not defined

In [6]:
# Searching for rows with have "BANY" and "NAS" in their county names (expect 0 rows)
fd_vio.query().searchByOperatorID("687080").searchByCounty("BAN").run()

NameError: name 'fd_vio' is not defined

In [8]:
# Just searching for "BANY" in COUNTY names
fd_vio.query().searchByCounty("BANY").run()

NameError: name 'fd_vio' is not defined

In [96]:
#Searching 3 at time, should return 0 rows
fd_vio.query().searchByCounty("BANY").searchByCounty("NAS").searchByCounty("Test").run()

SELECT * from food_service_inspections WHERE  "COUNTY" like %(county0)s AND  "COUNTY" like %(county1)s AND  "COUNTY" like %(county2)s


""
